# Entity Extraction - 2020 file was saved as 2021 by mistake

## Importing Libraries

In [2]:
import os
import requests
import re
import sys
import regex as re

In [3]:
import spacy

# Using Spacy Large
# !python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser"])

# Enable only NER
nlp.enable_pipe("ner")
nlp

2023-05-22 03:26:47.010868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 03:27:01.610556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-22 03:27:01.613809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
import nltk as nltk
import nltk.corpus  
from nltk.text import Text

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings("ignore")

In [6]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False, progress_bar=True)

Available CPUs: 16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [10]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [18]:
from tqdm import tqdm
from collections import Counter


In [9]:
def spacy_ner(x):
  doc = nlp(x)
  entities = []
  labels = []
  for ent in doc.ents:
    entities.append(ent.text)
    labels.append(ent.label_)
  entities_labels = list(zip(entities, labels)) #we do not want unique entities
  return entities_labels

**2020**

In [8]:
%%time
df_filt_2020=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2020_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2020.head(3)

CPU times: user 19 s, sys: 2.51 s, total: 21.5 s
Wall time: 1min 1s


,Unnamed: 0,url,date,language,title,text,cleaned_text,article_source,clean_title,title_tokens,...,year_pub,month,month-year,flag_relevant,num_tokens,rake_phrases_articles,rake_phrases_joined,final_topic,sent_probs,sent_label
0,0,https://fusionscienceacademy.com/artificial-in...,2020-01-30,en,Artificial Intelligence (AI) in Social Media ...,\n\nArtificial Intelligence (AI) in Social Med...,Artificial Intelligence AI in Social MediaMar...,2026 – Fusion Science Academy,Artificial Intelligence (AI) in Social Media ...,"['artificial', 'intelligence', 'ai', 'social',...",...,2020,1,Jan 2020,1,22283,['burkert fluid control systems emerson electr...,burkert fluid control systems emerson electric...,6,[[2.1574432e-04 9.9863547e-01 1.1488239e-03]],1
1,1,https://health.economictimes.indiatimes.com/ne...,2020-01-10,en,artificial intelligence: Researchers develop A...,\n\nartificial intelligence: Researchers devel...,artificial intelligence: Researchers develop ...,NaN,artificial intelligence: Researchers develop A...,"['artificial', 'intelligence', 'researchers', ...",...,2020,1,Jan 2020,1,8087,['economic times ethealthworldhome news hospit...,economic times ethealthworldhome news hospital...,1,[[1.9508268e-04 8.0524624e-04 9.9899966e-01]],2
2,2,https://heraldpublicist.com/bet-gil-on-ai-fina...,2020-01-15,en,Bet Gil on AI Final Fantasy Tactics Matches in...,\n\nBet Gil on AI Final Fantasy Tactics Matche...,Bet Gil on AI Final Fantasy Tactics Matches i...,Herald Publicist,Bet Gil on AI Final Fantasy Tactics Matches in...,"['bet', 'gil', 'ai', 'final', 'fantasy', 'tact...",...,2020,1,Jan 2020,1,4458,['hilarious twitch streamnewstechnologycricket...,hilarious twitch streamnewstechnologycricketpo...,1,[[5.5355614e-04 9.9602497e-01 3.4214482e-03]],1


In [11]:
%%time
tqdm.pandas()
df_filt_2020['entities_spacy'] = df_filt_2020['cleaned_text'].parallel_apply(lambda x: spacy_ner(x)).progress_apply(lambda x: x)

100%|██████████| 32667/32667 [00:00<00:00, 786346.25it/s]

CPU times: user 35.9 s, sys: 4.79 s, total: 40.7 s
Wall time: 8min 14s


In [ ]:
df_filt_2020['ENT_ORG'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2020['ENT_PROD'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2020['ENT_PER'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2020['ENT_NORP'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

In [14]:
%%time

##SAVED AS THE WRONG FILE NAME BY MISTAKE, CORRECT LATER
df_filt_2020.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments_NER.csv')

CPU times: user 35.5 s, sys: 525 ms, total: 36 s
Wall time: 1min 4s


In [20]:
from itertools import chain
df_filt_2020_ent_orgs = list(chain.from_iterable(df_filt_2020['ENT_ORG']))

counter = Counter(df_filt_2020_ent_orgs)
top_20 = counter.most_common(20)
for item, count in top_20:
    print(f"Organization: {item}, Count: {count}")

Organization: AI, Count: 37756
Organization: V19, Count: 28787
Organization: Google, Count: 20518
Organization: Microsoft, Count: 14155
Organization: IBM, Count: 13379
Organization: Artificial Intelligence, Count: 12495
Organization: GR, Count: 11260
Organization: Facebook, Count: 9600
Organization: Artificial Intelligence AI, Count: 7383
Organization: Amazon, Count: 6893
Organization: Machine Learning, Count: 6402
Organization: Intel, Count: 6191
Organization: Apple, Count: 5274
Organization: ML, Count: 4460
Organization: AsiaPacific, Count: 4086
Organization: Healthcare, Count: 3892
Organization: Samsung, Count: 3010
Organization: Twitter, Count: 2701
Organization: Oracle, Count: 2699
Organization: IBM Corporation, Count: 2665


In [21]:
from itertools import chain
df_filt_2020_ent_prods = list(chain.from_iterable(df_filt_2020['ENT_PROD']))

counter = Counter(df_filt_2020_ent_prods)
top_20 = counter.most_common(20)
for item, count in top_20:
    print(f"Products: {item}, Count: {count}")

Products: AI, Count: 70971
Products: V19, Count: 2601
Products: AIdriven, Count: 1194
Products: Twitter YouTube, Count: 1113
Products: JavaScript, Count: 984
Products: SE, Count: 718
Products: Galaxy, Count: 669
Products: YouTube, Count: 572
Products: TensorFlow, Count: 532
Products: Global AI, Count: 459
Products: HPC, Count: 443
Products: Discovery, Count: 433
Products: Javascript Course, Count: 422
Products: MasteryData ScienceData, Count: 422
Products: Facebook, Count: 390
Products: CRM, Count: 369
Products: A100, Count: 366
Products: FoodProduct, Count: 352
Products: Xilinx, Count: 332
Products: Windows 10, Count: 326


In [22]:
from itertools import chain
df_filt_2020_ent_pers = list(chain.from_iterable(df_filt_2020['ENT_PER']))

counter = Counter(df_filt_2020_ent_pers)
top_20 = counter.most_common(20)
for item, count in top_20:
    print(f"Person: {item}, Count: {count}")

Person: Size, Count: 3979
Person: Trump, Count: 3283
Person: Instagram, Count: 1360
Person: Middle EastAfrica, Count: 1277
Person: Biden, Count: 1046
Person: Richests Richests, Count: 1016
Person: Gebru, Count: 886
Person: Ai Weiwei, Count: 857
Person: Musk, Count: 808
Person: Elon Musk, Count: 779
Person: Trumps, Count: 776
Person: Busco, Count: 774
Person: asistencia tcnica para mi solucin de cuidados de salud Busco, Count: 774
Person: Donald Trump, Count: 704
Person: Pichai, Count: 647
Person: BFSI, Count: 613
Person: Kyle Wiggers, Count: 587
Person: Sundar Pichai, Count: 575
Person: Jim Thorpe, Count: 564
Person: Mark My ServicesAll, Count: 525


In [23]:
from itertools import chain
df_filt_2020_ent_norp = list(chain.from_iterable(df_filt_2020['ENT_NORP']))

counter = Counter(df_filt_2020_ent_norp)
top_20 = counter.most_common(20)
for item, count in top_20:
    print(f"NORP: {item}, Count: {count}")

NORP: Chinese, Count: 4344
NORP: American, Count: 1869
NORP: European, Count: 1783
NORP: British, Count: 1654
NORP: Canadian, Count: 1486
NORP: French, Count: 1077
NORP: Americans, Count: 860
NORP: German, Count: 847
NORP: Hindu, Count: 742
NORP: Tamil, Count: 541
NORP: Asian, Count: 513
NORP: ian, Count: 446
NORP: RemoteMoneyAll, Count: 444
NORP: African, Count: 437
NORP: Canadians, Count: 434
NORP: EUs, Count: 433
NORP: Irish, Count: 423
NORP: Democratic, Count: 398
NORP: Popular, Count: 393
NORP: North American, Count: 363


----
**2021**

In [24]:
df_filt_2021=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2021['entities_spacy'] = df_filt_2021['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))

In [ ]:
df_filt_2021['ENT_ORG'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2021['ENT_PROD'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2021['ENT_PER'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2021['ENT_NORP'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

In [26]:
%%time
df_filt_2021.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2021v2_Topics_Sentiments_NER.csv')

CPU times: user 46.5 s, sys: 808 ms, total: 47.3 s
Wall time: 1min 18s


In [27]:
def get_top_ents(df):
    org_list = list(chain.from_iterable(df['ENT_ORG']))
    
    
    
    prod_list = list(chain.from_iterable(df['ENT_PROD']))
    per_list = list(chain.from_iterable(df['ENT_PER']))
    norp_list = list(chain.from_iterable(df['ENT_NORP']))
    
counter = Counter(df_filt_2020_ent_norp)
top_20 = counter.most_common(20)
for item, count in top_20:
    print(f"NORP: {item}, Count: {count}")

SyntaxError: EOL while scanning string literal (2375784512.py, line 7)

------

In [36]:
%%time
df_filt_2022=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2022_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2022['entities_spacy'] = df_filt_2022['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))


CPU times: user 1min 20s, sys: 11.1 s, total: 1min 31s
Wall time: 13min 30s


In [37]:
%%time
df_filt_2023=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2023_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2023['entities_spacy'] = df_filt_2023['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))


CPU times: user 1min 22s, sys: 12.6 s, total: 1min 35s
Wall time: 14min 56s


### Analyzing NERs